In [55]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor

# TargetEncoder class for ModelID which will be used later as it has too many values to be used as a one hot encoded columns
class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, col, target):
        self.col = col
        self.target = target
        self.mapping_ = None
        self.global_mean_ = None

    def fit(self, X, y):
        df = pd.concat([X, y], axis=1)
        self.mapping_ = df.groupby(self.col)[self.target].mean()
        self.global_mean_ = y.mean()
        return self

    def transform(self, X):
        X = X.copy()
        X[self.col + "_encoded"] = X[self.col].map(self.mapping_)
        X[self.col + "_encoded"].fillna(self.global_mean_, inplace=True)
        return X

# Loading data
df_sales = pd.read_csv(r"C:\Adithya\IITK\IITK EDU\Stamatics- Deep Flow\datasheet\bluebook-for-bulldozers\TrainAndValid.csv", low_memory=False)
df_machine_info = pd.read_csv(r"C:\Adithya\IITK\IITK EDU\Stamatics- Deep Flow\datasheet\bluebook-for-bulldozers\Machine_Appendix.csv")

df_machine_info = df_machine_info.drop(columns=['MachineID']).drop_duplicates(subset='ModelID')
df = df_sales.merge(df_machine_info, on='ModelID', how='left')

#Extracting useful info from saledate that helps model depreciation/inflation from manufactured data
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['saleYear'] = df['saledate'].dt.year.astype(int)
df['saleMonth'] = df['saledate'].dt.month.astype(int)

# Handling NaNs, different strategies for different columns based on observation
mean_impute = ['MachineHoursCurrentMeter', 'PrimaryLower', 'PrimaryUpper']
none_impute = [
    'Forks', 'Pad_Type', 'Ride_Control', 'Turbocharged', 'Blade_Extension', 'Blade_Width',
    'Enclosure_Type', 'Scarifier', 'Tip_Control', 'Tire_Size', 'Coupler_System',
    'Grouser_Tracks', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer', 'Blade_Type',
    'ProductSize', 'Drive_System', 'Enclosure', 'Stick', 'Transmission', 'Ripper', 'Coupler',
    'Hydraulics_Flow', 'Track_Type', 'Grouser_Type', 'Travel_Controls', 'Steering_Controls', 'PrimarySizeBasis', 'Hydraulics', 'Differential_Type'
]
mode_impute = ['Engine_Horsepower', 'Pushblock', 'MfgYear', 'fiManufacturerID', 'fiManufacturerDesc']

for col in mean_impute:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

for col in none_impute:
    if col in df.columns:
        df[col] = df[col].fillna("None or Unspecified")

for col in mode_impute:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])

#Based on a possible relation between the 2, different strategy used to fill Nans of UsageBand
if 'UsageBand' in df.columns and 'fiProductClassDesc_x' in df.columns:
    df['UsageBand'] = df.groupby('fiProductClassDesc_x')['UsageBand'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'None'))

#Dropping redundant columns based on observations
df.drop(columns=[
    'saledate', 'SalesID', 'MachineID', 'datasource', 'auctioneerID',
    'fiSecondaryDesc_x', 'fiSecondaryDesc_y', 'fiBaseModel_x', 'fiBaseModel_y',
    'fiModelSeries_x', 'fiModelSeries_y', 'fiModelDescriptor_x', 'fiModelDescriptor_y',
    'Backhoe_Mounting', 'Blade_Type', 'fiModelDesc_x', 'fiModelDesc_y',
    'fiProductClassDesc_y', 'ProductGroup_y', 'ProductGroupDesc_y', 'fiManufacturerDesc'
], inplace=True)

# Preparing for encoding
target_col = 'SalePrice'
X = df.drop(columns=[target_col])
y = df[target_col]

# Splitting before fitting encoder
X_train, X_test, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Applying target encoder
te = TargetEncoder(col='ModelID', target='SalePrice')
te.fit(X_train, y_train)

X_train = te.transform(X_train)
X_test = te.transform(X_test)

# Dropping original ModelID
X_train.drop(columns=['ModelID'], inplace=True)
X_test.drop(columns=['ModelID'], inplace=True)

# Final feature separation
numeric_cols = X_train.select_dtypes(include=['int32', 'int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

# One-hot + scaling
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Training model
lr = LinearRegression()
lr.fit(X_train_transformed, y_train)

rf = RandomForestRegressor(n_estimators=1, random_state=42, n_jobs=-1)
rf.fit(X_train_transformed, y_train)

# Prediction
y_valid_pred_lr = lr.predict(X_test_transformed)
y_valid_pred_rf = rf.predict(X_test_transformed)

# Evaluation
print("RMSLE for linear regression model:", np.sqrt(metrics.mean_squared_log_error(np.abs(y_valid), np.abs(y_valid_pred_lr))))
print("RMSLE for random forest model:", np.sqrt(metrics.mean_squared_log_error(np.abs(y_valid), np.abs(y_valid_pred_rf))))

#Test Data
test_df = pd.read_csv(r"C:\Adithya\IITK\IITK EDU\Stamatics- Deep Flow\datasheet\bluebook-for-bulldozers\Test.csv")
test_df = test_df.merge(df_machine_info, on='ModelID', how='left')


test_df['saledate'] = pd.to_datetime(test_df['saledate'], errors='coerce')
test_df['saleYear'] = test_df['saledate'].dt.year.astype(int)
test_df['saleMonth'] = test_df['saledate'].dt.month.astype(int)
Ids = test_df['SalesID']
test_df.drop(columns=[
    'saledate', 'SalesID', 'MachineID', 'datasource', 'auctioneerID',
    'fiSecondaryDesc_x', 'fiSecondaryDesc_y', 'fiBaseModel_x', 'fiBaseModel_y',
    'fiModelSeries_x', 'fiModelSeries_y', 'fiModelDescriptor_x', 'fiModelDescriptor_y',
    'Backhoe_Mounting', 'Blade_Type', 'fiModelDesc_x', 'fiModelDesc_y',
    'fiProductClassDesc_y', 'ProductGroup_y', 'ProductGroupDesc_y', 'fiManufacturerDesc'
], inplace=True)


for col in mean_impute:
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna(df[col].mean())
for col in none_impute:
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna("None or Unspecified")
for col in mode_impute:
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna(df[col].mode()[0])

if 'UsageBand' in test_df.columns and 'fiProductClassDesc_x' in test_df.columns:
    test_df['UsageBand'] = test_df.groupby('fiProductClassDesc_x')['UsageBand'].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'None'))

test_df=te.transform(test_df)
test_df.drop(columns=['ModelID'],inplace=True)

X = preprocessor.transform(test_df)
test_preds_lr = lr.predict(X)
test_preds_rf = rf.predict(X)

df_benchmark=pd.read_csv(r"C:\Adithya\IITK\IITK EDU\Stamatics- Deep Flow\datasheet\bluebook-for-bulldozers\random_forest_benchmark_test.csv")
bench_pred=df_benchmark['SalePrice']

print("RMSE for test data for linear regression model in comparison with benchmark:", np.sqrt(metrics.mean_squared_log_error(np.abs(test_preds_lr), np.abs(bench_pred))))
print("RMSE for test data for random forest in comparison with the benchmark:", np.sqrt(metrics.mean_squared_log_error(np.abs(test_preds_rf), np.abs(bench_pred))))
# Save submission of rf cause it is marginally performing better than lr
submission = pd.DataFrame({"SalesID": Ids, "SalePrice": test_preds_rf})
submission.to_csv("submission.csv", index=False)

C:\Users\Venkat Raman\AppData\Local\Temp\ipykernel_21760\2536131177.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[self.col + "_encoded"].fillna(self.global_mean_, inplace=True)
C:\Users\Venkat Raman\AppData\Local\Temp\ipykernel_21760\2536131177.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

RMSLE for linear regression model: 0.3528842843861324
RMSLE for random forest model: 0.2792275099376999
RMSE for test data for linear regression model in comparison with benchmark: 0.4933582544174968
RMSE for test data for random forest in comparison with the benchmark: 0.2651757357892384


C:\Users\Venkat Raman\AppData\Local\Temp\ipykernel_21760\2536131177.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[self.col + "_encoded"].fillna(self.global_mean_, inplace=True)
